In [1]:
import pandas as pd
import numpy as np

import yaml

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
                            recall_score, f1_score, log_loss, precision_recall_curve

import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier

import optuna

from typing import Tuple, Set

import warnings
warnings.filterwarnings('ignore')

RAND=42

N_FOLDS = 4

KeyboardInterrupt: 

In [ ]:
pd.options.mode.chained_assignment = None
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
def extract_words(string):
    return list(map(float, string[1:-1].split()))

In [ ]:
def get_metrics(y_test, y_pred, y_score, name = "Default"):
    """Метрики для задачи классификации"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
    df_metrics['Precision'] = precision_score(y_test, y_pred, zero_division=0)
    df_metrics['Recall'] = recall_score(y_test, y_pred, zero_division=0)
    df_metrics['f1'] = f1_score(y_test, y_pred, zero_division=0)
    df_metrics['Logloss'] = log_loss(y_test, y_score)

    return df_metrics

In [ ]:
config_path = "../config/params.yaml"
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

recommender = config["recommeder"]

# Baseline

In [ ]:
df = pd.read_csv("eda.csv")
df[:5]

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df['vectorized_tokens'] = df['vectorized_tokens'].apply(extract_words)

In [ ]:
df = df.astype(recommender['change_type_columns'])

In [ ]:
df.info()

In [12]:
for i in tqdm(range(100)):
    df[str(i)] = df['vectorized_tokens'].apply(lambda x: x[i])

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
df = df.drop(columns=['vectorized_tokens', 'n_unique_okpd2'])

In [14]:
df[:5]

,purchase,min_publish_date,forsmallbiz,price,customer,supplier,is_winner,reg_code,month,flag_won,...,90,91,92,93,94,95,96,97,98,99
0,3,2021-02-02,1,290000.00,2,1,1,58.2_2,2,0,...,-0.001322,-0.030157,0.008215,0.000441,-0.014258,-0.024656,-0.002359,-0.015422,0.008751,-0.004480
1,2834,2021-02-02,1,105000.00,218,1,1,62.0_2,2,0,...,0.017369,-0.009209,0.011130,0.017798,-0.026846,-0.041504,-0.004734,0.010884,0.019125,0.027479
2,154949,2021-02-16,1,98967.50,11235,1,1,63.9_2,2,0,...,0.013283,-0.014217,-0.038393,0.038132,-0.030937,-0.000821,0.002925,-0.010350,-0.014901,0.005936
3,154949,2021-02-16,1,98967.50,11235,1650,0,63.9_2,2,0,...,0.013283,-0.014217,-0.038393,0.038132,-0.030937,-0.000821,0.002925,-0.010350,-0.014901,0.005936
4,147950,2021-03-10,0,77460.03,11061,1,1,62.0_2,3,0,...,0.024791,-0.025200,0.019539,0.004568,-0.032321,0.002326,0.014077,0.035959,0.004383,0.052171


In [15]:
# удаляем колонки, которые не известны при рекомендациях
df = df.drop(columns=['flag_won', 'is_winner', 'purchase_size'])

In [16]:
# определяем train выборку
df_train = df[df['min_publish_date'] < '2022-11-01']
df_train[:5]

,purchase,min_publish_date,forsmallbiz,price,customer,supplier,reg_code,month,0,1,...,90,91,92,93,94,95,96,97,98,99
0,3,2021-02-02,1,290000.00,2,1,58.2_2,2,0.099951,-0.045924,...,-0.001322,-0.030157,0.008215,0.000441,-0.014258,-0.024656,-0.002359,-0.015422,0.008751,-0.004480
1,2834,2021-02-02,1,105000.00,218,1,62.0_2,2,0.101108,-0.044075,...,0.017369,-0.009209,0.011130,0.017798,-0.026846,-0.041504,-0.004734,0.010884,0.019125,0.027479
2,154949,2021-02-16,1,98967.50,11235,1,63.9_2,2,0.096125,-0.075322,...,0.013283,-0.014217,-0.038393,0.038132,-0.030937,-0.000821,0.002925,-0.010350,-0.014901,0.005936
3,154949,2021-02-16,1,98967.50,11235,1650,63.9_2,2,0.096125,-0.075322,...,0.013283,-0.014217,-0.038393,0.038132,-0.030937,-0.000821,0.002925,-0.010350,-0.014901,0.005936
4,147950,2021-03-10,0,77460.03,11061,1,62.0_2,3,0.181689,-0.079811,...,0.024791,-0.025200,0.019539,0.004568,-0.032321,0.002326,0.014077,0.035959,0.004383,0.052171


In [17]:
# определяем test выборку
df_test = df[df['min_publish_date'] >= '2022-11-01']
df_test[:5]

,purchase,min_publish_date,forsmallbiz,price,customer,supplier,reg_code,month,0,1,...,90,91,92,93,94,95,96,97,98,99
26,1064,2022-11-15,1,181720.00,70,1,58.2_2,11,0.118955,-0.058594,...,0.000105,-0.022247,0.005643,-0.004025,-0.013740,0.003093,0.004564,0.006269,-0.006186,0.010823
27,1729,2022-12-06,1,167448.00,105,1,62.0_2,12,0.102236,-0.078738,...,0.007899,-0.019196,-0.042672,0.069824,-0.028125,-0.061662,0.021649,-0.020416,-0.027389,-0.014978
28,2430,2022-12-08,1,200248.16,201,1,62.0_2,12,0.133927,-0.085102,...,0.003507,-0.026729,-0.053180,0.071892,-0.040827,-0.037405,0.033203,-0.001161,-0.021750,-0.004136
29,156898,2022-12-12,0,190740.00,11266,1,62.0_2,12,0.069835,-0.065892,...,0.011013,-0.015165,-0.055468,0.081177,-0.033349,-0.020764,0.015203,0.006279,-0.005878,-0.007706
308,412017,2022-11-07,1,2886156.00,9651,100009,27.4_77,11,0.102239,-0.057709,...,0.024234,0.007161,0.032809,0.006940,-0.025554,-0.008215,0.024814,0.015174,0.010784,-0.048401


In [18]:
# определяем поставщиков, которые есть в test, но отсутствуют в train
sup_to_remove = set(df_test['supplier'].unique()) - set(df_train['supplier'].unique())

In [19]:
df_submission = df_test.groupby('supplier')['purchase'].apply(set).to_frame(name='purchases')
# удаляем поставщиков, которые появились на платформе в последние 2 месяца
df_submission = df_submission.drop(sup_to_remove, axis=0)
df_submission[:5]

,purchases
supplier,
1,"{1064, 1729, 156898, 2430}"
2,"{141478, 536503}"
3,{149878}
4,{}
5,{154789}


In [20]:
# удаляем поставщиков, которых нет на test
df_submission = df_submission.loc[df_submission['purchases'] != set()]
df_submission[:5]

,purchases
supplier,
1,"{1064, 1729, 156898, 2430}"
2,"{141478, 536503}"
3,{149878}
5,{154789}
6,"{41633, 539818, 481931, 40178, 443698, 535989,..."


In [21]:
def extract_less_4(row):
    return df_train[df_train['supplier'] == row.name].shape[0] < 4

# выделим поставщиков df_submission, содержащих менее 4 участий на train
df_submission_less_4 = df_submission[df_submission.apply(extract_less_4, axis=1)]
df_submission_less_4[:5]

,purchases
supplier,
160,{415}
178,"{153344, 145730, 147043, 141508, 149879, 16401..."
244,{408}
263,{151070}
290,{3052}


In [22]:
# удаляем поставщиков с небольшим кол-вом участий из df_submission
df_submission = df_submission.drop(set(df_submission_less_4.index), axis=0)

In [23]:
def find_closest_vectors(target_vectors: np.ndarray, vector_space: np.ndarray, 
                         k: int =5) -> np.ndarray:
    """
    Find the k vectors in the given vector_space that are closest to the given target_vectors.

    Args:
        target_vectors (numpy.ndarray): A 2D numpy array of shape (n, m) 
            where n is the number of target vectors and m is the dimensionality of the vectors.
        vector_space (numpy.ndarray): A 2D numpy array of shape (p, m) 
            where p is the number of vectors in the vector space and m is the 
            dimensionality of the vectors.
        k (int): The number of closest vectors to return. Default is 5.

    Returns:
        numpy.ndarray: A 1D numpy array of length k containing the indices of 
        the closest vectors in the vector space.
    """
    sum_distances = []
    
    
    mms = MinMaxScaler()
    
    
    #Нормализуем векторы
    target_vectors = mms.fit_transform(target_vectors)
    vector_space = mms.transform(vector_space)
    
    
    for vector in vector_space:
        # создаем массив расстояний между вектором в пространстве и целевыми векторами
        distances = np.linalg.norm(target_vectors - vector, axis=1)
        # сохраняем сумму расстояний между вектором в пространстве и целевыми векторами
        sum_distances.append(np.sum(distances))
    
    
    # выбираем набор векторов с минимальной суммой расстояний до каждого вектора в target_vectors
    if k >= vector_space.shape[0]:
        return np.nonzero(vector_space)
    
    
    return np.argpartition(sum_distances, k)[:k]

In [24]:
# определяем столбцы для удаления перед обучением
columns_to_drop = ['supplier', 'min_publish_date', 'reg_code']

In [25]:
def baseline_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, params = None) -> pd.DataFrame:
    """
    Trains a LightGBM classifier on the training set and returns the score on the test set.

    Args:
        df_tr (pandas.DataFrame): A pandas DataFrame containing the training set.
        df_t (pandas.DataFrame): A pandas DataFrame containing the test set.

    Returns:
        pandas.DataFrame: The score on the test set.

    """

    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
    

    model = LGBMClassifier(random_state=RAND, class_weight='balanced', n_jobs=-1)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    y_score = model.predict_proba(x_test)

    return get_metrics(y_test, y_pred, y_score)

In [26]:
def edit_data(df_train: pd.DataFrame, df_test: pd.DataFrame, 
              df: pd.DataFrame, sup: int) -> Tuple[pd.DataFrame, pd.DataFrame, Set]:
    """
    The function filters the training and test datasets of a supplier with a given 
    reg_code and returns the filtered dataframes as well as the set of participations 
    of this supplier in the original dataset.

    Args:

        df_train: pandas.DataFrame - the training dataset
        df_test: pandas.DataFrame - the test dataset
        df: pandas.DataFrame - the original dataset
        sup: str - the registration code of the supplier

    Returns:

        Tuple[pandas.DataFrame, pandas.DataFrame, set] - a tuple of the filtered 
        dataframes of the training and test datasets, as well as the set of participations 
        of the supplier.
    """
    unique_reg_okpd = df_train[df_train['supplier'] == sup]['reg_code'].unique()
    
    # фильтруем train и test на основе уникальных reg_code поставщиков 
    df_sup_train = df_train[df_train['reg_code'].isin(unique_reg_okpd)]
    df_sup_test = df_test[df_test['reg_code'].isin(unique_reg_okpd)]
    
    # выделяем все участия поставщика
    df_sup_true = set(df[df['supplier'] == sup]['purchase'])
    
    # проверяем, остались ли участия на test после фильтрации
    if df_sup_test['purchase'].isin(df_sup_true).nunique() < 2:
        df_sup_test = df_test.copy()
        
    # удаляем выделенные столбцы и дубликаты
    df_sup_train = df_sup_train.drop(columns=columns_to_drop).drop_duplicates()
    df_sup_test = df_sup_test.drop(columns=columns_to_drop).drop_duplicates()
    
    df_sup_test = df_sup_test.set_index('purchase')
    df_sup_train = df_sup_train.set_index('purchase')
    
    return df_sup_train, df_sup_test, df_sup_true

In [27]:
def rec_algorithm(df_train: pd.DataFrame, df_test: pd.DataFrame, 
                  df_submission: pd.DataFrame, df: pd.DataFrame) -> pd.DataFrame:
    """
    Recommends items for the first 500 suppliers in df_submission based on 
    training data in df_train and df.
    Uses the LightGBM classifier and the baseline_lgbm function to make predictions.

    Args:
    - df_train (pd.DataFrame): Training data with columns 'purchase', 'supplier', 'reg_code', and 'target'.
    - df_test (pd.DataFrame): Test data with columns 'purchase', 'supplier', 'reg_code'.
    - df_submission (pd.DataFrame): Dataframe with the first 500 suppliers for which to recommend items.
    - df (pd.DataFrame): Dataframe with columns 'purchase', 'supplier', 'reg_code'.

    Returns:
    - metrics (pd.DataFrame): Mean of the ROC AUC score for each supplier in 
    df_submission after running the baseline_lgbm function.
    """
    metrics = pd.DataFrame()

    for sup in tqdm(df_submission.index[:500]):

        # подготавливаем данные к обучению
        df_sup_train, df_sup_test, df_sup_true = edit_data(df_train, df_test, df, sup)

        # добавляем метки для обучения алгоритма классификации 
        df_sup_train['target'] = df_sup_train.index.isin(df_sup_true).astype(int)
        df_sup_test['target'] = df_sup_test.index.isin(df_sup_true).astype(int)

        metrics = pd.concat([metrics, baseline_lgbm(df_sup_train, df_sup_test)])

    return metrics.mean(axis=0)

In [28]:
def alg_nearest(df_train: pd.DataFrame, df_test: pd.DataFrame, 
                df_submission_less_4: pd.DataFrame, df: pd.DataFrame) -> float:
    """
    Computes the average recall score for the top 500 rows of df_submission_less_4 
    using the nearest neighbor algorithm.

    Args:
    - df_train: training data DataFrame
    - df_test: testing data DataFrame
    - df_submission_less_4: submission data DataFrame with purchase information
    - df: DataFrame containing information about the suppliers

    Returns:
    - float: the average recall score for the top 500 rows of df_submission_less_4
    """

    recall_less_4 = []

    for sup in tqdm(df_submission_less_4.index[:500]):
        # подготавливаем данные к обучению
        df_sup_train, df_sup_test, df_sup_true = edit_data(df_train, df_test, df, sup)

        # выделяем векторы участия поставщика на train
        vectors = df_sup_train[df_sup_train.index.isin(df_sup_true)]

        # определяем векторы наиболее близки для поставщика на test
        idx = find_closest_vectors(vectors, df_sup_test)
        y_pred = set(df_sup_test.iloc[idx].index)

        recall = len(y_pred & df_submission_less_4.loc[sup]['purchases']) \
                 / len(df_submission_less_4.loc[sup]['purchases'])

        recall_less_4.append(recall)

    return (np.mean(recall_less_4))

In [87]:
rec_algorithm(df_train, df_test, df_submission, df)

  0%|          | 0/500 [00:00<?, ?it/s]

/tmp/ipykernel_19588/1381042027.py:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return metrics.mean(axis=0)


Accuracy     0.956422
ROC_AUC      0.841957
Precision    0.241158
Recall       0.266748
f1           0.219859
Logloss      0.196172
dtype: float64

In [103]:
alg_nearest(df_train, df_test, df_submission_less_4, df)

  0%|          | 0/500 [00:00<?, ?it/s]

0.3411196863246043

# Tune

In [28]:
def objective(trial: optuna.Trial, x: pd.DataFrame, y: pd.Series, learning_rate=None) -> float:
    """
    This function defines the objective function for an Optuna study to tune hyperparameters
    for a LightGBM binary classification model. 

    Args:
        trial (optuna.Trial): A trial corresponding to a set of hyperparameters.
        x (pd.DataFrame): The features to be used for training and validation.
        y (pd.Series): The target variable for training and validation.

    Returns:
        float: The mean of the cross-validation AUC-ROC scores for the given set of hyperparameters.
    """
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [400]),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
#         'learning_rate': trial.suggest_categorical('learning_rate', [learning_rate]),
#         'max_bin': trial.suggest_int('max_bin', 200, 800),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 150),
#         'max_depth': trial.suggest_int('max_depth', 4, 12),
#         'min_child_samples': trial.suggest_int('min_child_samples', 200, 20000, step=200),
#         'lambda_l1': trial.suggest_int('lambda_l1', 0, 50),
#         'lambda_l2': trial.suggest_int('lambda_l2', 0, 50),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0.001, 0.1),
#         'objective': 'binary',
#         'metric': 'auc',
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.3, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.3, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 2, 6),
#         'random_state': RAND,
    }

    cv_pred = np.empty(N_FOLDS)
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):
        x_train_, x_val_ = x.iloc[train_idx], x.iloc[test_idx]
        y_train_, y_val_ = y.iloc[train_idx], y.iloc[test_idx]

        pruning = optuna.integration.LightGBMPruningCallback(trial, 'auc')

        model = LGBMClassifier(
            class_weight='balanced',
            n_jobs=-1,
            **params
        )
        model.fit(x_train_, y_train_,
                  eval_metric='auc',
                  eval_set=[(x_val_, y_val_)],
                  early_stopping_rounds=100,
                  callbacks=[pruning],
                  verbose=0)

        y_pred = model.predict(x_val_)
        y_proba = model.predict_proba(x_val_)[:, 1]

        cv_pred[fold] = roc_auc_score(y_val_, y_proba)
    return (np.mean(cv_pred))

In [33]:
def tune_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, learning_rate=None) -> pd.DataFrame:
    
    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
     
        
    func = lambda trial: objective(trial, x_train, y_train, learning_rate)

    study = optuna.create_study(direction="maximize")
    study.optimize(func, n_trials=50, n_jobs=-1)
    
    return study.best_params

In [32]:
def open_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)
    
def save_file(file_path, data):       
    with open(file_path, 'w') as file:
        yaml.dump(data, file)

In [31]:
def tune_algorithm(df_train: pd.DataFrame, df_test: pd.DataFrame, 
                   df_submission: pd.DataFrame, df: pd.DataFrame, sup, learning_rate=None):
    
    # подготавливаем данные к обучению
    df_sup_train, df_sup_test, df_sup_true = edit_data(df_train, df_test, df, sup)

    # добавляем метки для обучения алгоритма классификации 
    df_sup_train['target'] = df_sup_train.index.isin(df_sup_true).astype(int)
    df_sup_test['target'] = df_sup_test.index.isin(df_sup_true).astype(int)

    params = tune_lgbm(df_sup_train, df_sup_test, sup, learning_rate)
        
    return params

In [30]:
file_path = "../config/model_params.yaml"

In [34]:
data = open_file(file_path)

for sup in tqdm(df_submission.index[:500]):

    if sup in data:
        params = tune_algorithm(df_train, df_test, df_submission, df, sup, **data[sup])
        data[sup].extend(params)
        
    # Если ключа еще нет, создать новый список
    else:
        params = tune_algorithm(df_train, df_test, df_submission, df, sup)        
        data[sup] = params

save_file(file_path, data)

  0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=8, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8
[LightGBM] [Warning] bagging_fraction is set=0.6751165316026022, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6751165316026022
[LightGBM] [Warning] feature_fraction is set=0.4683249767096229, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4683249767096229
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] lambda_l1 is set=6, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6
[LightGBM] [Warning] bagging_fraction is set=0.5035541546690397, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5035541546690397
[LightGBM] [Warning] feature_fraction is set=0.535969077444137, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.53596907

[LightGBM] [Warning] lambda_l1 is set=36, reg_alpha=0.0 will be ignored. Current value: lambda_l1=36
[LightGBM] [Warning] bagging_fraction is set=0.6920323854263122, subsample=1.0 will be ignored. Current value: bagging_fraction

limit_output extension: Maximum message size of 10000 exceeded with 14658 characters

In [29]:
def classifier_lgbm(df_tr: pd.DataFrame, df_t: pd.DataFrame, params = None) -> pd.DataFrame:
    """
    Trains a LightGBM classifier on the training set and returns the score on the test set.

    Args:
        df_tr (pandas.DataFrame): A pandas DataFrame containing the training set.
        df_t (pandas.DataFrame): A pandas DataFrame containing the test set.

    Returns:
        pandas.DataFrame: The score on the test set.

    """

    x_train = df_tr[df_tr.columns[:-1]]
    y_train = df_tr['target']
    
    x_test = df_t[df_tr.columns[:-1]]
    y_test = df_t['target']
    

    model = LGBMClassifier(class_weight='balanced', n_jobs=-1, **params)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    y_score = model.predict_proba(x_test)

    return get_metrics(y_test, y_pred, y_score)

In [33]:
data = open_file(file_path)

In [35]:
metrics = pd.DataFrame()

for sup in tqdm(df_submission.index[:500]):

    # подготавливаем данные к обучению
    df_sup_train, df_sup_test, df_sup_true = edit_data(df_train, df_test, df, sup)

    # добавляем метки для обучения алгоритма классификации 
    df_sup_train['target'] = df_sup_train.index.isin(df_sup_true).astype(int)
    df_sup_test['target'] = df_sup_test.index.isin(df_sup_true).astype(int)
    
    params = data[sup]

    metrics = pd.concat([metrics, classifier_lgbm(df_sup_train, df_sup_test, params=params)])

  0%|          | 0/500 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l1 is set=36, reg_alpha=0.0 will be ignored. Current value: lambda_l1=36
[LightGBM] [Warning] bagging_fraction is set=0.6920323854263122, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6920323854263122
[LightGBM] [Warning] lambda_l2 is set=23, reg_lambda=0.0 will be ignored. Current value: lambda_l2=23
[LightGBM] [Warning] feature_fraction is set=0.3616062238831329, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3616062238831329
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l1 is set=19, reg_alpha=0.0 will be ignored. Current value: lambda_l1=19
[LightGBM] [Warning] bagging_fraction is set=0.5560994109885059, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5560994109885059
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
[LightGBM] [Warning] feature_fraction is

[LightGBM] [Warning] lambda_l1 is set=11, reg_alpha=0.0 will be ignored. Current value: lambda_l1=11
[LightGBM] [Warning] bagging_fraction is set=0.44222680312989393, subsample=1.0 will be ignored. Current value: bagging_fraction=0.44222680312989393
[LightGBM] [Warning] lambda_l2 is set=12, reg_lambda=0.0 will be ignored. Current value: lambda_l2=12
[LightGBM] [Warning] feature_fraction is set=0.931552869556695, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.931552869556695
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] bagging_fraction is set=0.9917167081050979, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9917167081050979
[LightGBM] [Warning] lambda_l2 is set=42, reg_lambda=0.0 will be ignored. Current value: lambda_l2=42
[LightGBM] [Warning] feature_fraction 

limit_output extension: Maximum message size of 10000 exceeded with 10400 characters

In [36]:
metrics.mean(axis=0)

Accuracy     0.730974
ROC_AUC      0.743162
Precision    0.090660
Recall       0.638708
f1           0.139746
Logloss      0.436611
dtype: float64